In [30]:
import datetime
import sys

sys.path.append("..")

from src.auth.aurora import get_sales_client, get_portal_client

In [31]:
client_name = "ADT Fire & Security Ltd"

In [32]:
sales_client = get_sales_client()
sales_client_id = sales_client.query_single_value(
    """SELECT client_id
    FROM sales.t_client
    WHERE name = %(client_name)s""",
    {"client_name": client_name},
)

In [33]:
subscriptions_df = sales_client.query_to_pandas(
    """SELECT
        s.device_id,
        s.subscription_type,
        s.start_date,
        s.expiry_date,
        cancel.cancel_date,
        (
                CASE WHEN s.pay_frequency = "Monthly" THEN s.unit_price
                ELSE s.unit_price / s.contract_term
                END
        ) AS mrr
    FROM sales.t_subscription AS s
    LEFT JOIN sales.t_orders AS o
    USING(order_number)
    LEFT JOIN sales.t_cancellation AS cancel
    ON s.subscription_id = cancel.subscription_id
    WHERE item_code IN ('101-104-01', '101-104-02', '101-204-01', '101-204-02')
    AND o.client_id = %(client_id)s
    """,
    params={"client_id": sales_client_id},
)

In [34]:
test_date = datetime.date(2025, 6, 23)
subs_subset_df = subscriptions_df[(
    (subscriptions_df["start_date"] <= test_date)
    & (
        (subscriptions_df["expiry_date"] > test_date)
        | (subscriptions_df["expiry_date"].isnull())
    )
    & (
        (subscriptions_df["cancel_date"] > test_date)
        | (subscriptions_df["cancel_date"].isnull())
    )
)]

In [45]:
subs_devices = set(subs_subset_df["device_id"].unique())

In [46]:
client_name = "ADT Fire & Security PLC"

In [47]:
portal_client = get_portal_client()

root_client_id = portal_client.query_single_value(
    """SELECT id
    FROM api_clients
    WHERE name = %(client_name)s
    AND deleted_at IS NULL
    AND parent_id = 1""",
    {"client_name": client_name},
)

client_list = portal_client.query_to_list(
    """WITH RECURSIVE category_path AS (
    SELECT id, name, parent_id
    FROM api_clients
    WHERE parent_id = %(root_client_id)s
    AND deleted_at IS NULL

    UNION ALL

    SELECT c.id, c.name, c.parent_id
    FROM api_clients c
    INNER JOIN category_path cp ON cp.id = c.parent_id
    WHERE deleted_at IS NULL
    )
    SELECT DISTINCT id
    FROM category_path
    """,
    {"root_client_id": root_client_id},
)
client_list.append(root_client_id)

placeholders = ', '.join(['%s'] * len(client_list))
portal_devices_df = portal_client.query_to_pandas(
    f"""SELECT
        d.id,
        d.device_serial,
        d.imei,
        d_t.type,
        d.client_id,
        d.active,
        MIN(s.created_at) AS created_at,
        MAX(s.expires_at) AS expires_at
    FROM api_devices AS d
    LEFT JOIN api_subscriptions AS s
    ON s.id = d.subscription_id
    LEFT JOIN api_device_types AS d_t
    ON d.device_type_id = d_t.id
    WHERE d.client_id IN ({placeholders})
    AND d.device_type_id IN (0, 1, 14, 15)
    AND device_serial IS NOT NULL
    AND s.created_at IS NOT NULL
    GROUP BY d.id, d.device_serial, d_t.type, d.client_id""",
    params=client_list
)

In [ ]:
portal_devices_df["created_at"]

0      2015-02-17
1      2015-02-17
2      2015-02-17
3      2015-02-25
4      2020-12-04
          ...    
135    2025-01-23
136    2025-01-23
137    2025-01-23
138    2025-02-17
139    2025-06-23
Name: created_at, Length: 140, dtype: object

In [59]:
portal_devices_df[(
    (portal_devices_df["created_at"].dt.date <= test_date)
    & (
        (portal_devices_df["expires_at"].dt.date > test_date)
        | (portal_devices_df["expires_at"].isnull())
    )
)]

,id,device_serial,imei,type,client_id,active,created_at,expires_at
7,22219,G01-00954,0a10aced202194944a051670,Nimbus Go,33,1,2024-11-26,2025-12-31 23:59:59
8,22220,G01-00955,0a10aced202194944a051540,Nimbus Go,33,1,2024-11-26,2025-12-31 23:59:59
9,24063,43090,352890064929837,Nimbus Gateway,33,1,2025-03-10,2026-03-31 23:59:59
10,25492,C01-00807,0a10aced202194944a0756a8,Nimbus Connect,33,1,2025-06-04,2026-06-30 23:59:59
11,25493,C01-00808,0a10aced202194944a0756e0,Nimbus Connect,33,1,2025-06-04,2026-06-30 23:59:59
...,...,...,...,...,...,...,...,...
133,23307,43035,352890064924499,Nimbus Gateway,24532,1,2025-01-23,2026-01-31 23:59:59
134,23308,43036,352890064918392,Nimbus Gateway,24532,1,2025-01-23,2026-01-31 23:59:59
135,23314,43037,352890064912536,Nimbus Gateway,24532,1,2025-01-23,2026-01-31 23:59:59
136,23315,43038,352890064914631,Nimbus Gateway,24532,1,2025-01-23,2026-01-31 23:59:59


In [60]:
portal_devices = set(portal_devices_df["device_serial"].unique())

In [61]:
subs_devices - portal_devices

{'41871',
 '42570',
 '42571',
 '42572',
 '42573',
 '42574',
 '42575',
 '42576',
 '42577',
 '42578',
 '42579',
 '42580',
 '42581',
 '42582',
 '42583',
 '42584',
 '42586',
 '42587',
 '42588',
 '42590',
 '42591',
 '42593',
 '42594',
 '42595',
 '42596',
 '42597',
 '42598',
 '42599',
 '42601',
 '42602',
 '42603',
 '42606',
 '42607',
 '42608',
 '42609',
 '42610',
 '42611',
 '42612',
 '42613',
 '42614',
 '42615',
 '42616',
 '42618',
 '42619',
 '42620',
 '42622',
 '42625',
 '42627',
 '42628',
 '42629',
 '42630',
 '42631',
 '42632',
 '42633'}

In [50]:
portal_client = get_portal_client()

In [51]:
client_list = portal_client.query_to_list(
    """WITH RECURSIVE category_path AS (
    SELECT id, name, parent_id
    FROM api_clients
    WHERE parent_id = %(root_client_id)s
    AND deleted_at IS NULL

    UNION ALL

    SELECT c.id, c.name, c.parent_id
    FROM api_clients c
    INNER JOIN category_path cp ON cp.id = c.parent_id
    WHERE deleted_at IS NULL
    )
    SELECT DISTINCT id
    FROM category_path
    """,
    {"root_client_id": root_client_id},
)
client_list.append(root_client_id)

In [52]:
placeholders = ', '.join(['%s'] * len(client_list))

portal_devices_df = portal_client.query_to_pandas(
    f"""SELECT
        d.id,
        d.device_serial,
        d.imei,
        d_t.type,
        d.client_id,
        d.active,
        MIN(s.created_at) AS created_at,
        MAX(s.expires_at) AS expires_at
    FROM api_devices AS d
    LEFT JOIN api_subscriptions AS s
    ON s.id = d.subscription_id
    LEFT JOIN api_device_types AS d_t
    ON d.device_type_id = d_t.id
    WHERE d.client_id IN ({placeholders})
    AND d.device_type_id IN (0, 1, 14, 15)
    AND device_serial IS NOT NULL
    AND s.created_at IS NOT NULL
    GROUP BY d.id, d.device_serial, d_t.type, d.client_id""",
    params=client_list
)